<p>
    <img align= left width=120 src="https://photos.angel.co/startups/i/8376586-be95fe56adf287171e22c21ac9d9f331-medium_jpg.jpg?buster=1623229867" >
    &nbsp; <h1> - Contextual Search Task</h1>
</p>

In [115]:
!jt -t grade3 -fs 11 -ofs 10

## EXPLORING DATASET

In [17]:
import pandas as pd

In [18]:
!ls

Untitled.ipynb           Zevi_search_dataset.json laptops.csv


In [19]:
df = pd.read_json("Zevi_search_dataset.json")

In [20]:
df.columns

Index(['name', 'description', 'brand', 'categories', 'hierarchicalCategories',
       'type', 'price', 'price_range', 'image', 'url', 'free_shipping',
       'rating', 'popularity', 'objectID'],
      dtype='object')

In [21]:
df.head()

,name,description,brand,categories,hierarchicalCategories,type,price,price_range,image,url,free_shipping,rating,popularity,objectID
0,Amazon - Fire TV Stick with Alexa Voice Remote...,"Enjoy smart access to videos, games and apps w...",Amazon,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,39.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5477...,https://api.bestbuy.com/click/-/5477500/pdp,False,4,21469,5477500
1,Google - Chromecast - Black,Google Chromecast: Enjoy a world of entertainm...,Google,"[TV & Home Theater, Streaming Media Players]","{'lvl0': 'TV & Home Theater', 'lvl1': 'TV & Ho...",Streaming media plyr,35.00,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/4397...,https://api.bestbuy.com/click/-/4397400/pdp,False,4,21468,4397400
2,"Dell - Inspiron 15.6"" Touch-Screen Laptop - In...",Dell Inspiron Laptop: Get speed and performanc...,Dell,"[Computers & Tablets, Laptops]","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Burst skus,499.99,200 - 500,https://cdn-demo.algolia.com/bestbuy-0118/5588...,https://api.bestbuy.com/click/-/5588602/pdp,True,4,21467,5588602
3,Amazon - Echo Dot,Deliver your favorite playlist anywhere in you...,Amazon,[Smart Home],{'lvl0': 'Smart Home'},Voice assistants,49.99,1 - 50,https://cdn-demo.algolia.com/bestbuy-0118/5578...,https://api.bestbuy.com/click/-/5578851/pdp,True,4,21466,5578851
4,"Apple - MacBook Air® (Latest Model) - 13.3"" Di...","MacBook Air features up to 8GB of memory, a fi...",Apple,"[Computers & Tablets, Laptops, All Laptops, Ma...","{'lvl0': 'Computers & Tablets', 'lvl1': 'Compu...",Apple,999.99,500 - 2000,https://cdn-demo.algolia.com/bestbuy-0118/6443...,https://api.bestbuy.com/click/-/6443034/pdp,True,4,21465,6443034


## INDEXING WITH ELASTIC SEARCH

In [8]:
!pip install elasticsearch

In [222]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [223]:
for index in es.indices.get('*'):
  print(index)

.apm-agent-configuration
.apm-custom-link
.kibana-event-log-7.13.3-000001
.kibana_7.13.3_001
.kibana_task_manager_7.13.3_001
test
zevi


/Users/lord/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: this request accesses system indices: [.apm-agent-configuration, .apm-custom-link, .kibana_7.13.3_001, .kibana_task_manager_7.13.3_001], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [11]:
import json
json_file = open('Zevi_search_dataset.json')
products = json.load(json_file) # load as json
for product in products : 
    res = es.index(index='zevi', doc_type="product", id=product['objectID'], body=product) # index in ES
    if res['_shards']['failed']==1 : # if indexing fails
        print(res) # print where did it fail
print("DONE")

/Users/lord/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


DONE


### WHAT THE CONTEXTUAL SEARCH SHOULD LOOK LIKE

In [10]:
query = 'Dell laptop 8GB RAM under 500'
search_term = 'laptop 8GB RAM' # query minus detected entities
price = 500 
brand = "Dell"
body = {"query": {"bool": {"must":  # If-else is needed acc to detected entities
      [
        { "match": {"name": search_term}   },
        { "match": { "brand": brand }      }
      ],
      "filter": [
        { "range": { "price": { "lte": price }}} # filter results to show less than 500$
      ]              
                          }}}
res = es.search(index='zevi', body=body)
for doc in res['hits']['hits']:
    print(doc['_source']['name'],f" : {int(doc['_source']['price'])}$""\n")

Dell - Inspiron 15.6" Laptop - Intel Pentium - 8GB Memory - 1TB Hard Drive - Black  : 449$

Dell - Inspiron 17.3" Laptop - AMD A9-Series - 8GB Memory - 1TB Hard Drive - Gray  : 499$

Dell - Latitude 14.1" Refurbished Laptop - Intel Core i5 - 8GB Memory - 500GB Hard Drive - Black  : 226$

Dell - Latitude 12.5" Refurbished Laptop - Intel Core i5 - 8GB Memory - 750GB Hard Drive - Black  : 269$

Dell - Latitude 14" Refurbished Laptop - Intel Core i5 - 8GB Memory - 500GB Hard Drive - Gray  : 310$

Dell - Latitude 12.5" Refurbished Laptop - Intel Core i5 - 8GB Memory - 750GB Hard Drive - Black  : 307$

Dell - Latitude 14" Refurbished Laptop - Intel Core i5 - 8GB Memory - 750GB Hard Drive - Black  : 325$

Dell - Latitude 13.3" Refurbished Laptop - Intel Core i5 - 8GB Memory - 750GB Hard Drive - Black  : 336$

Dell - Latitude 14" Refurbished Laptop - Intel Core i5 - 8GB Memory - 240GB Solid State Drive  : 363$

Dell - Inspiron 15.6" Touch-Screen Laptop - Intel Core i5 - 8GB Memory - 1TB Hard D

Also worth checking out : https://www.elastic.co/guide/en/elasticsearch/reference/1.4/search-explain.html

### NER : The Missing Hero !
<img src="https://i.pinimg.com/originals/98/26/ee/9826eec1919e6e616f8316ce69731b0f.jpg" width=500 align=left>

In [12]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = "Dell laptop 8GB RAM under 500"
doc = nlp(text)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

8 CARDINAL


#### Since Spacy is upto no good for our task, a custom NER is needed.
#### I will train my model using monkeylearn and use the resultant API here.

## Creating Dataset for custom NER

In [62]:
df['name'][1]

'Google - Chromecast - Black'

In [44]:
df['name'][1]+' - free shipping' + f" - {df['rating'][1]} star - {df['price'][1]}" # concat name

'Google - Chromecast - Black - free shipping - 4 star - 35.0'

In [119]:
import nltk
import re
from nltk.corpus import stopwords
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

stop_words = stopwords.words("english")

def preprocess(text, stopword=False):
    
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    if stopword :
        tokens = []
        for token in text.split():
            if token not in stop_words:
                tokens.append(token)
        return " ".join(tokens)
    else :
        return text

In [122]:
for i in [11,22,33,44,55]:
    print(preprocess(df['name'][i],stopword=True))

roku express streaming media player black
lenovo 110 15isk 15 6 laptop intel core i3 4gb memory 1tb hard drive black
philips hue white color ambiance a19 add smart led bulb 3rd gen multicolor
hp 61 2 pack ink cartridges black cyan magenta yellow
apple macbook air latest model 13 3 display intel core i5 8gb memory 256gb flash storage silver


In [123]:
for i in [11,22,33,44,55]:
    print(preprocess(df['name'][i],stopword=False))

roku express streaming media player black
lenovo 110 15isk 15 6 laptop intel core i3 4gb memory 1tb hard drive black
philips hue white and color ambiance a19 add on smart led bulb 3rd gen multicolor
hp 61 2 pack ink cartridges black cyan magenta yellow
apple macbook air latest model 13 3 display intel core i5 8gb memory 256gb flash storage silver


#### Both methods return very similar results for our dataset
This means our dataset has negligible stopwords. Therefore, there should not be any stopwords in user's search query. Hence, I will use **stopword=True** for searching.

In [128]:
name_df = pd.DataFrame(columns=['name'])
for i in range(len(df['name'])):
    name = preprocess(df['name'][i], stopword=True)
    if df['free_shipping'][i]: # If free shipping exists, save it in name
        name_df = name_df.append({'name': name +' free shipping' + f" {df['rating'][i]} star {int(df['price'][i])}$"}, ignore_index=True)
    else :
        name_df = name_df.append({'name': name + f" {df['rating'][i]} star {int(df['price'][i])}"}, ignore_index=True)
name_df.head()

,name
0,amazon fire tv stick alexa voice remote black ...
1,google chromecast black 4 star 35
2,dell inspiron 15 6 touch screen laptop intel c...
3,amazon echo dot free shipping 4 star 49
4,apple macbook air latest model 13 3 display in...


In [129]:
name_df.to_csv('training_data-processed.csv')
!ls

Untitled.ipynb              training_data-processed.csv
Zevi_search_dataset.json    training_data.csv
laptops.csv


In [130]:
len(name_df.name.unique().tolist())

21326

In [78]:
!pip install monkeylearn

In [135]:
from monkeylearn import MonkeyLearn

text = "HP Laptops 512GB 4 star 500$"
ml = MonkeyLearn('c2cf077e8886eecf879de6fff30e1f72b38652ef')
model_id = 'ex_sUxkptEt'
data = [text]
response = ml.extractors.extract(model_id, data=data)

In [136]:
for i in response.body[0]['extractions']:
    print(i)

{'tag_name': 'Brand', 'extracted_text': 'HP', 'parsed_value': 'HP', 'offset_span': [0, 1]}
{'tag_name': 'Rating', 'extracted_text': '4 star', 'parsed_value': '4 star', 'offset_span': [17, 22]}
{'tag_name': 'Price', 'extracted_text': '500', 'parsed_value': '500', 'offset_span': [24, 26]}


## Trying a manual explicit approach for Price Filter
- NER is not good to detect price
- Manual approach is needed
    - find amount reference
    - find whether query wants higher or lower
    - use it to specify price filters in ES query

In [176]:
import re
string = 'hello my name is vibhansh gupta'
pattern = 'vibhansh'
match=(re.search(pattern, string))
print(match.span())

(17, 25)


In [177]:
string[match.span()[0]:match.span()[1]]

'vibhansh'

In [350]:
def is_float(n):
    try:
        float(n)
    except ValueError:
        return False
    return True
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    return True

def amount(string):
    for i in string.split():
        if "$" or " $" in i :  # Find '$' in query
            i=i.replace("$","")
            return i
            break
        elif is_float(string.split()[0]) or is_int(string.split()[0]):
            return string.split()[0]
        else :
            return None
        
def price_filter(string):        
    greater = ['more than','above','over','higher than'] # filter keywords
    smaller = ['less than','below','under','lower than'] # filter keywords
    lte, gte, lte_amount, gte_amount=False, False, None, None
    for i in string.split() :
        for j in greater:
            if i == j : # if keyword found in query
                gte = True
                pos = re.search(i,string).span() # Search for position of keyword
                gte_amount = amount(string[pos[1]:]) # Find amount ahead of the keyword 
                break
        for j in smaller :
            if i == j : # if keyword found in query
                lte = True
                pos = re.search(i,string).span() # Search for position of keyword
                lte_amount = amount(string[pos[1]:]) # Find amount ahead of the keyword 
                break
    return { 'gte_flag':gte, 'gte_amount':gte_amount, 'lte_flag':lte, 'lte_amount':lte_amount}

In [352]:
l = "Dell laptop 512 GB with free shipping under 500.00 $ but more than 250"
res = price_filter(l)
print(res)

{'gte_flag': False, 'gte_amount': None, 'lte_flag': True, 'lte_amount': '500.00'}


# RUNNING THE SEARCH

In [279]:
"hello my name is abcd".replace("my name is","")

'hello  abcd'

In [392]:
import datetime
query = 'pink iPhone cover'
print("Given Text : ",query)
search_term = query
prices = price_filter(query) # Detect Price
print("\nPrices : ",prices)

# Run NER Model
data = [query]
response = ml.extractors.extract(model_id, data=data)

brand, colour, shipping, rating, latest = None, None, None, None, None
print("\n ENTITIES DETECTED :")
for i in response.body[0]['extractions']:
    print('\n',i)
    search_term = search_term.replace(i['extracted_text'],"") # Remove unnecessary stuff
    # Save every entity if found
    if i['tag_name']=='Brand':
        brand=i['extracted_text']
    elif i['tag_name']=='Colour':
        colour=i['extracted_text']
    elif i['tag_name']=='Shipping':
        shipping=i['extracted_text']
    elif i['tag_name']=='Rating':
        rating=i['extracted_text'][0]
for i in query.split():
    if i.lower()=='latest' :
        search_term = search_term.replace(i,"")
        latest = str(datetime.date.today().year)

# Remove amount found from search term
if prices['gte_flag']:
    search_term = search_term.replace(prices['gte_amount']+"$","")
if prices['lte_flag']:
    search_term = search_term.replace(prices['lte_amount']+"$","")
    
search_term = preprocess(search_term, stopword=True)
print('\nSearch Term Extracted:',search_term)

Given Text :  pink iPhone cover

Prices :  {'gte_flag': False, 'gte_amount': None, 'lte_flag': False, 'lte_amount': None}

 ENTITIES DETECTED :

 {'tag_name': 'Colour', 'extracted_text': 'pink', 'parsed_value': 'pink', 'offset_span': [0, 3]}

Search Term Extracted: iphone cover


In [393]:
print(brand, colour, shipping, rating, latest)

None pink None None None


In [396]:
# AVAILABLE FILTERS : BRAND, COLOUR, FREE SHIPPING, RATING & PRICE RANGE
filters = []
if prices['gte_flag']:
    filters.append({ "range": { "price": { "gte": prices['gte_amount'] }}})
if prices['lte_flag']:
    filters.append({ "range": { "price": { "lte": prices['lte_amount'] }}})
if brand is not None :
    filters.append({ "match": { "brand": brand }})
if rating is not None :
    filters.append({ "term": { "rating": rating }})
if shipping is not None :
    filters.append({ "term": { "free_shipping": True }})
    
musts = [{ "match": {"name": search_term} }]
if colour is not None:
    musts.append({ "match": {"name": colour} })
if latest is not None:
    musts.append({ "match": {"name": latest} })
# if [brand, colour, shipping, rating, latest] == [None, None, None, None, None]:
#     musts.append({ "match": {"description": search_term}})

In [397]:
print("QUERY WAS : ", query,"\n")
body = {"query": {"bool": {"must": musts, # to show must needed results
                           "filter": filters # to show relevant results
                           }     }    }     

res = es.search(index='zevi', body=body)
for doc in res['hits']['hits']:
    print("NAME : ",doc['_source']['name']," RATING : ",doc['_source']['rating'],
                "PRICE : ",doc['_source']['price'],"\n")

QUERY WAS :  pink iPhone cover 

NAME :  OtterBox - Defender Series Protective Cover for Apple iPhone SE - Pink  RATING :  4 PRICE :  49.99 

NAME :  ADOPTED - Back Cover for Apple iPhone 6 and 6s - Pink  RATING :  4 PRICE :  39.99 

NAME :  Hello Kitty - Polycarbonate Cover for Apple® iPhone® 5 - Black/Pink  RATING :  3 PRICE :  19.99 

NAME :  Incipio - PERFORMANCE Back Cover for Samsung Galaxy S7 - Pink  RATING :  5 PRICE :  24.99 

NAME :  Incipio - STOWAWAY Back Cover for Samsung Galaxy S7 - Pink  RATING :  4 PRICE :  27.99 

NAME :  Apple - Smart Cover for iPad mini 4 - Pink Sand  RATING :  4 PRICE :  39.99 

NAME :  Apple - Smart Cover for iPad mini 4 - Light Pink  RATING :  4 PRICE :  39.99 

NAME :  Apple - Smart Cover for 9.7-inch iPad Pro - Pink Sand  RATING :  4 PRICE :  49.99 

NAME :  Apple - Silicone Cover for 9.7-inch iPad Pro - Light Pink  RATING :  4 PRICE :  69.99 

NAME :  Apple - Silicone Cover for 9.7-inch iPad Pro - Pink Sand  RATING :  4 PRICE :  69.99 



In [373]:
latest

'2021'